#  Introductions to constrainat-based modeling using cobrapy

## Part 2: Flux Balance Analysis

### Instructor:
* Miguel Ponce de León from (Barcelona Supercomputing Center)
* Contact: miguel.ponce@bsc.es

11 December, 2020

# Part 2: Genome-scale modelling.

In this part we are gonna use a genome-scale metabolic model of Escherichia coli named iJO1366
The file has already been stored in the data folder and its path is data/iJO1366.xml

Alternatively, you can also access it here:
- [http://bigg.ucsd.edu/models/iJO1366](http://bigg.ucsd.edu/models/iJO1366)

to download the model and to see other metadata (citation, description, etc)

## Part 2.1: Studying the model.

### Read the SBML model

First we need to import the function read_sbml_model from the cobra.io modules

In [44]:
import cobra
from cobra.io import read_sbml_model

# State the path to the file iJO1366.xml
sbml_fname = './data/iJO1366.xml'

# Read the model
model = read_sbml_model(sbml_fname)

## Exercise 2.1: Inspecting the model's numbers

How many metabolites genes and reactions are contained in the model?

In [2]:
## TODO
## Write your code below
model

Name,iJO1366
Memory address,749c70e9a410
Number of metabolites,1805
Number of reactions,2583
Number of genes,1367
Number of groups,36
Objective expression,1.0*Ec_biomass_iJO1366_WT_53p95M - 1.0*Ec_biomass_iJO1366_WT_53p95M_reverse_55db7
Compartments,"Cytoplasm, Extracellular, Periplasm"


In [7]:
# Inspecting the genes
# 1. Access a particular reaction:
#    * You can do it directly with: gene = model.genes.b0720
#    * Or you can use the function get_by_id: gene = model.genes.get_by_id('b0720')

gene = model.genes.b0720

gene = model.genes.get_by_id('b0722')

gene.name
# 6. Inspect the reaction by printing:
#    1. gene.name
#    2. gene.id
#    3. gene.reactions

'b0722'

### Inspecting the systems' boundaries

* see the exchange fluxes
* see the objective function (the reaction set to be optimized)

Use print(model.summary())

You can also find the objective function using the following filtering technique:
* [r for r in model.reactions if r.objective_coefficient == 1]
* the reaction id of the biomass is Ec_biomass_iJO1366_WT_53p95M
and the exchange fluxes can be accessed using:
* model.boundary

In [9]:
## TODO
## Write your code below
model.reactions.DM_4CRSOL

Reaction identifier,DM_4CRSOL
Name,Sink needed to allow p-Cresol to leave system
Memory address,0x749c70b34fa0
Stoichiometry,4crsol_c --> p-Cresol -->
GPR,
Lower bound,0.0
Upper bound,1000.0


### Running a Flux Balance Analysis (FBA).

Documentation: [https://cobrapy.readthedocs.io/en/latest/simulating.html](https://cobrapy.readthedocs.io/en/latest/simulating.html)

By default, the model boundary condition (growth medium) is M9 aerobic (glucose minimal)

1.  Check the medium by inspecting the lower_bound of the following reactions:
  * EX\_glc\_e\_.lower_bound
  * EX\_o2\_e\_.lower_bound
2.  Optimize biomass using: 
  * solution = model.optimize()
  
3.  Inspect the solution as we did previously in Part 1.2 Optimization.



In [10]:
solution = model.optimize()

print("Objective value: %.2f\n" % solution.objective_value)
print("Status: %s\n" % solution.status)

print("Fluxes:\n")
print(solution.fluxes)

# Converting the solution into a pandas dataframe
df = solution.to_frame()
# Saving the solution into tab-separed-value (tsv) format (plain text)
# df.to_csv("out/iJO1366_fba.tsv", sep="\t")

Objective value: 0.96

Status: optimal

Fluxes:

DM_4CRSOL    0.000215
DM_5DRIB     0.000223
DM_AACALD    0.000000
DM_AMOB      0.000002
DM_MTHTHF    0.001293
               ...   
ZN2abcpp     0.000000
ZN2t3pp      0.000000
ZN2tpp       0.000313
ZNabcpp      0.000000
Zn2tex       0.000313
Name: fluxes, Length: 2583, dtype: float64


Inspect the flux value of the following reactions
* The glucose consumption: EX_glc_e_
* The oxygen consumption: EX_o2_e_
* The growth rate: Ec_biomass_iJO1366_WT_53p95M

HINT 1: use the solution object -> solution.fluxes.reaction_id <br>
HINT 2: use model.summary()

In [19]:
## TODO
## Write your code below
model.reactions.EX_o2_e_.lower_bound = 0
model.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
ca2_e,EX_ca2_e_,0.0006737,0,0.00%
cbl1_e,EX_cbl1_e_,3.034E-05,62,0.00%
cl_e,EX_cl_e_,0.0006737,0,0.00%
cobalt2_e,EX_cobalt2_e_,3.265E-06,0,0.00%
cu2_e,EX_cu2_e_,9.17E-05,0,0.00%
fe2_e,EX_fe2_e_,0.001102,0,0.00%
fe3_e,EX_fe3_e_,0.001011,0,0.00%
glc_D_e,EX_glc_e_,10,6,100.00%
k_e,EX_k_e_,0.02526,0,0.00%
mg2_e,EX_mg2_e_,0.001123,0,0.00%


## Exercise 2.2: 

1. Change the oxygen exchange lower bound to zero to simulate anaerobic growth.
2. Optimize the model
3. What is the maximal growth rate in anaerobic conditions
4. what are the main three secretion products?

In [28]:
for r in model.metabolites.lac_D_c.reactions:
    print(r)

LDH_D: lac_D_c + nad_c <=> h_c + nadh_c + pyr_c
ACM6PH: acmum6p_c + h2o_c --> acgam6p_c + lac_D_c
LDH_D2: lac_D_c + q8_c --> pyr_c + q8h2_c
GLYOX3: h2o_c + mthgxl_c --> h_c + lac_D_c
D_LACt2pp: h_p + lac_D_p <=> h_c + lac_D_c
GLYOX: h2o_c + lgt_S_c --> gthrd_c + h_c + lac_D_c


In [36]:
## TODO
## Write your code below
model.reactions.EX_o2_e_.lower_bound = 0
import json
with model:
    model.reactions.ACKr.knock_out()
    model.reactions.LDH_D.knock_out()
    solution = model.optimize()
    print(model.summary())
    df = solution.to_frame()
    df.to_csv("ferment_etoh_fba.tsv", sep="\t")
    df["fluxes"].to_json("ferment_etoh_fba.json")
    



Objective
1.0 Ec_biomass_iJO1366_WT_53p95M = 0.1360495589996054

Uptake
------
Metabolite      Reaction      Flux  C-Number  C-Flux
     ca2_e     EX_ca2_e_ 0.0006737         0   0.00%
    cbl1_e    EX_cbl1_e_ 3.034E-05        62   0.00%
      cl_e      EX_cl_e_ 0.0006737         0   0.00%
 cobalt2_e EX_cobalt2_e_ 3.265E-06         0   0.00%
     cu2_e     EX_cu2_e_  9.17E-05         0   0.00%
     fe2_e     EX_fe2_e_  0.001102         0   0.00%
     fe3_e     EX_fe3_e_  0.001011         0   0.00%
   glc_D_e     EX_glc_e_        10         6 100.00%
       k_e       EX_k_e_   0.02526         0   0.00%
     mg2_e     EX_mg2_e_  0.001123         0   0.00%
     mn2_e     EX_mn2_e_ 8.952E-05         0   0.00%
    mobd_e    EX_mobd_e_ 1.864E-05         0   0.00%
     nh4_e     EX_nh4_e_     1.429         0   0.00%
     ni2_e     EX_ni2_e_ 4.177E-05         0   0.00%
      pi_e      EX_pi_e_    0.1261         0   0.00%
     so4_e     EX_so4_e_   0.03428         0   0.00%
     zn2_e     EX_zn

## Exercise 2.3: 

1. Set the oxygen exchange lower bound to -20
2. Set the glucose exchange flux (EX_glc_D_e_ lower bound to 0)
3. Set the glucose exchange flux (EX_ac_e_ lower bound to -10)

What is the maximal growth rate using acetate as soley carbon source
what is the oxygen uptake rate?

In [55]:
## TODO
## Write your code below
model.reactions.EX_o2_e_.lower_bound = -3
model.reactions.EX_glc_e_.lower_bound = 0

model.reactions.ATPM.lower_bound = 


model.reactions.EX_ac_e_.lower_bound = -10
model.optimize()
model.summary()



Metabolite,Reaction,Flux,C-Number,C-Flux
ac_e,EX_ac_e_,1.698,2,100.00%
ca2_e,EX_ca2_e_,4.578E-05,0,0.00%
cbl1_e,EX_cbl1_e_,2.061E-06,62,0.00%
cl_e,EX_cl_e_,4.578E-05,0,0.00%
cobalt2_e,EX_cobalt2_e_,2.219E-07,0,0.00%
cu2_e,EX_cu2_e_,6.231E-06,0,0.00%
fe2_e,EX_fe2_e_,7.487E-05,0,0.00%
fe3_e,EX_fe3_e_,6.867E-05,0,0.00%
h_e,EX_h_e_,1.616,0,0.00%
k_e,EX_k_e_,0.001717,0,0.00%


## Optional Visualizing flux distributions using Escher

[Escher documentation](https://escher.readthedocs.io/en/latest/)

Escher online WebApp: [https://escher.github.io/](https://escher.github.io/#/)

In [ ]:
import escher
from escher import Builder

# Lets crate a builder by passing our model as well a given map name to tell escher how to represent the network
# Check the escher web to see other maps https://escher.github.io/#/
builder = Builder(organism='Escherichia coli', map_name='iJO1366.Central metabolism')
builder.reaction_data = solution.fluxes
builder

In [ ]:
# Add the optimal flux distribution to our map builder
model.reactions.EX_o2_e_.lower_bound = -20
model.reactions.EX_glc_e_.lower_bound = -10
model.reactions.EX_ac_e_.lower_bound = 0


gene =  model.genes.b0720
with model:
    gene.knock_out()
    ko_solution = model.optimize()

builder.reaction_data = ko_solution.fluxes
builder